In [2]:
pip install --upgrade --user hmmlearn

Requirement already up-to-date: hmmlearn in c:\users\haris\appdata\roaming\python\python37\site-packages (0.2.3)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import pandas as pd
from random import seed,random
from random import random
from hmmlearn import hmm


In [5]:
def get_multinorm(multi):
    r=np.random.uniform(0.0, 1.0)
    CS=np.cumsum(multi)
    CS=np.insert(CS,0,0)
    m=(np.where(CS<r))[0]
    nextState=m[len(m)-1]
    return nextState

In [6]:
P = np.array([[0.4, 0.3, 0.3],
              [0.2, 0.6, 0.2],
              [0.1, 0.1, 0.8]])

state=np.array([[0.5,0.2,0.3]])

currentState=0
stateHist=state
dfStateHist=pd.DataFrame(state)
distr_hist = [[0,0,0]]

for x in range(1000):
    currIn=np.ma.masked_values((P[currentState]), 0.0)
    nextState=get_multinorm(currIn)
    state[0,nextState]=1.0
    stateHist=np.append(stateHist,state,axis=0)
    currentState=nextState
    totals=np.sum(stateHist,axis=0)
    gt=np.sum(totals)
    distrib=totals/gt
    distrib=np.reshape(distrib,(1,3))
    distr_hist=np.append(distr_hist,distrib,axis=0)

dfDistrHist = pd.DataFrame(distr_hist)

In [7]:
dfDistrHist

,0,1,2
0,0.000000,0.000000,0.000000
1,0.277778,0.555556,0.166667
2,0.277778,0.555556,0.166667
3,0.324675,0.519481,0.155844
4,0.350000,0.500000,0.150000
...,...,...,...
996,0.333859,0.334362,0.331779
997,0.333858,0.334361,0.331781
998,0.333858,0.334360,0.331783
999,0.333857,0.334359,0.331784


In [8]:
model = hmm.GaussianHMM(n_components=3, covariance_type="full")
model.startprob_ = np.array([0.5,0.2,0.3])
model.transmat_ = np.array([[0.4, 0.3, 0.3],
                            [0.2, 0.6, 0.2],
                            [0.1, 0.1, 0.8]])
model.means_ = np.array([[0.0, 0.0], [3.0, -3.0], [5.0, 10.0]])
model.covars_ = np.tile(np.identity(2), (3, 1, 1))
X, Z = model.sample(1000)


In [9]:
X,Z

(array([[ 0.785883  , -0.80471983],
        [-0.16168513,  1.2253538 ],
        [ 4.36027134,  8.17539291],
        ...,
        [ 3.32565395, -3.66560754],
        [ 3.40451488, -3.39370882],
        [-1.41090817,  0.4149104 ]]),
 array([0, 0, 2, 2, 2, 1, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1,
        1, 1, 1, 2, 0, 0, 2, 2, 2, 0, 0, 2, 1, 0, 2, 2, 2, 2, 2, 2, 1, 1,
        2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1,
        2, 2, 2, 2, 2, 0, 0, 1, 0, 0, 1, 1, 0, 2, 2, 2, 1, 0, 0, 1, 0, 0,
        0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 1, 2, 2, 2,
        2, 2, 1, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 0, 0, 1, 2, 2, 2, 1, 1,
        2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 0,
        0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 1, 2, 2, 0, 1, 1, 1, 1, 2,
        2, 2, 2, 2, 2, 2, 2, 

# Using the data generated estimate P,A and compare with the parameters used togenerate the data

In [15]:
def transition_matrix(transitions):
    n = 1+ max(transitions) #number of states

    M = [[0]*n for _ in range(n)]

    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

#test:

t = Z
m = transition_matrix(t)
print("Estimated transition matrix A")
for row in m: print(' '.join('{0:.2f}'.format(x) for x in row))

Estimated transition matrix A
0.38 0.28 0.34
0.19 0.58 0.23
0.09 0.10 0.81


From the above the estimated transition matrix is similar to given transition matrix 

In [42]:
count_s1 = 0
count_s2 = 0
count_s3 = 0
i = 0
while i < 1000:
    if (Z[i] == 0):
        count_s1+=1
    elif (Z[i] == 1):
        count_s2+=1
    else:
        count_s3+=1
    i = i + 3
print("PI")
PI = [count_s1/100,count_s2/100,count_s3/100]
PI





PI


[0.48, 0.22, 0.3]

# References: 
    
https://stackoverflow.com/questions/46657221/generating-markov-transition-matrix-in-python

https://towardsdatascience.com/markov-chain-analysis-and-simulation-using-python-4507cee0b06e